In [163]:
from itertools import product
from operator import itemgetter

from PIL import Image
import random
import numpy

import torch

from torch.utils.data import Dataset, DataLoader
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision.transforms import ToTensor

numpy.set_printoptions(threshold=numpy.nan)

In [2]:
## step 1: create test and train sets

In [3]:
def create_image(x, y):
    im = Image.new(mode='L', size=(128, 128), color=0)
    pixels = im.load()
    pixels[x, y] = 256
    return im

In [4]:
data = [(x, y) for x,y in product(range(128), range(128))]

In [5]:
random.shuffle(data)

In [6]:
train_data = data[:int(len(data)*0.8)]
test_data = data[int(len(data)*0.8):]

In [7]:
for x,y in train_data:
    im = create_image(x, y)
    im.save("./train_set/{}_{}.bmp".format(x, y))

for x,y in test_data:
    im = create_image(x, y)
    im.save("./test_set/{}_{}.bmp".format(x, y))

In [8]:
## step 2: train the network to find radius of circle [only use train data in this stage]

In [9]:
def rotated_img(image, center):
    images = numpy.array([numpy.array(image.rotate(d, center=center)) for d in range(360)])
    arr = numpy.array(numpy.sum(images, axis=(0)), dtype=numpy.uint8)
    out = Image.fromarray(arr, mode='L')
    return out

In [147]:
class Circles(Dataset):
    def __init__(self, train_data):
        self.train_xy = train_data
        self.to_tensor = ToTensor()
    def __len__(self):
        return len(self.train_xy)
    def __getitem__(self, i):
        x, y = self.train_xy[i]
        im = Image.open("./train_set/{}_{}.bmp".format(x, y))
        #center = (random.randint(0, 127), random.randint(0, 127))
        center = random.sample([(0, 0), (128, 0), (0, 128), (128, 128)], 1)
        rotated_image = rotated_img(im, center[0])
        radius = ((x - center[0][0])**2 + (y - center[0][1])**2)**0.5
        return {
            "image": self.to_tensor(rotated_image),
            "radius": torch.LongTensor([int(radius / 185.0 * 100.0)])
        }

In [148]:
dataset = Circles(train_data)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)

In [149]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 10, 5)
        self.conv3 = nn.Conv2d(10, 10, 3)
        self.fc1 = nn.Linear(10 * 13 * 13, 250)
        self.fc2 = nn.Linear(250, 200)
        self.fc3 = nn.Linear(200, 100)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 10 * 13 * 13)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [150]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# net.to(device)

In [151]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [153]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs
        inputs = data["image"]
        radius = data["radius"].view(data["radius"].numel())
        #inputs, radius = inputs.to(device), radius.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, radius)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
            print(radius[0], torch.max(outputs, 1)[1][0])

print('Finished Training')

[1,   100] loss: 4.537
tensor(68) tensor(68)
[1,   200] loss: 4.508
tensor(63) tensor(68)
[1,   300] loss: 4.441
tensor(24) tensor(68)
[1,   400] loss: 4.387
tensor(79) tensor(60)
[1,   500] loss: 4.390
tensor(49) tensor(58)
[1,   600] loss: 4.351
tensor(10) tensor(62)
[1,   700] loss: 4.350
tensor(82) tensor(68)
[1,   800] loss: 4.387
tensor(66) tensor(60)
[2,   100] loss: 4.347
tensor(34) tensor(61)
[2,   200] loss: 4.327
tensor(63) tensor(66)
[2,   300] loss: 4.267
tensor(51) tensor(60)
[2,   400] loss: 4.190
tensor(45) tensor(57)
[2,   500] loss: 3.863
tensor(48) tensor(51)
[2,   600] loss: 3.121
tensor(60) tensor(60)
[2,   700] loss: 2.583
tensor(50) tensor(51)
[2,   800] loss: 2.204
tensor(64) tensor(64)
[3,   100] loss: 1.887
tensor(28) tensor(28)
[3,   200] loss: 1.705
tensor(57) tensor(56)
[3,   300] loss: 1.589
tensor(67) tensor(67)
[3,   400] loss: 1.513
tensor(54) tensor(54)
[3,   500] loss: 1.410
tensor(76) tensor(75)
[3,   600] loss: 1.368
tensor(65) tensor(65)
[3,   700]

Process Process-2326:
Process Process-2327:
Exception ignored in: <bound method Image.__del__ of <PIL.Image.Image image mode= size=0x0 at 0x7F169E81FD68>>
Process Process-2328:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/shrey/.virtualenvs/nn-tests/lib/python3.6/site-packages/PIL/Image.py", line 602, in __del__
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    if (hasattr(self, 'fp') and hasattr(self, '_exclusive_fp')
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/shrey/.virtualenvs/nn-tests/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106

KeyboardInterrupt: 

Process Process-2325:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/shrey/.virtualenvs/nn-tests/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 911, in wait
    ready = selector.select(timeout)
  File "/usr/lib/python3.6/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout

In [187]:
GUESSES = [(x, y) for x, y in product(range(128), range(128))]
GUESS_R = [
    (
        ((x - 0) ** 2 + (y - 0) ** 2) ** 0.5,
        ((x - 128) ** 2 + (y - 0) ** 2) ** 0.5,
        ((x - 0) ** 2 + (y - 128) ** 2) ** 0.5
    )
    for x, y in GUESSES   
]

def get_r(im, net, center):
    to_tensor = ToTensor()
    rotated_image = to_tensor(rotated_img(im, center))
    out = net(rotated_image.unsqueeze(0))
    r = torch.max(out, 1)[1][0]
    return r.item() * 1.0 / 100.0 *  185.0

def get_x_y(im, net):
    r1 = get_r(im, net, (0, 0))
    r2 = get_r(im, net, (128, 0))
    r3 = get_r(im, net, (0, 128))
    guess_dist = [
        (r1 - r1_guess) ** 2 + (r2 - r2_guess) ** 2 + (r3 - r3_guess) ** 2
        for r1_guess, r2_guess, r3_guess in GUESS_R
    ]
    idx = min(enumerate(guess_dist), key=itemgetter(1))[0]
    return GUESSES[idx]


In [191]:
import glob

for fname in glob.glob("./test_set/*.bmp"):
    im = Image.open(fname)
    x,y = get_x_y(im, net)
    real_x, real_y = fname.split("/")[-1].split(".")[0].split("_")
    print("REAL = {}, {} .... GUESSED = {}, {}".format(real_x, real_y, x, y))


REAL = 51, 7 .... GUESSED = 51, 8
REAL = 16, 30 .... GUESSED = 15, 30
REAL = 107, 92 .... GUESSED = 107, 92
REAL = 54, 46 .... GUESSED = 53, 47
REAL = 110, 96 .... GUESSED = 110, 95
REAL = 43, 0 .... GUESSED = 44, 0
REAL = 37, 126 .... GUESSED = 36, 125
REAL = 82, 17 .... GUESSED = 82, 18
REAL = 49, 81 .... GUESSED = 48, 81
REAL = 127, 26 .... GUESSED = 71, 78
REAL = 91, 72 .... GUESSED = 90, 71
REAL = 38, 108 .... GUESSED = 37, 107
REAL = 127, 89 .... GUESSED = 127, 89
REAL = 94, 72 .... GUESSED = 93, 71
REAL = 8, 57 .... GUESSED = 9, 56
REAL = 84, 94 .... GUESSED = 84, 93
REAL = 27, 4 .... GUESSED = 28, 5
REAL = 122, 19 .... GUESSED = 71, 74
REAL = 111, 95 .... GUESSED = 112, 95
REAL = 87, 24 .... GUESSED = 86, 23
REAL = 22, 37 .... GUESSED = 23, 37
REAL = 28, 93 .... GUESSED = 27, 92
REAL = 15, 61 .... GUESSED = 15, 60
REAL = 101, 113 .... GUESSED = 100, 113
REAL = 103, 102 .... GUESSED = 102, 101
REAL = 32, 44 .... GUESSED = 33, 44
REAL = 90, 98 .... GUESSED = 90, 97
REAL = 28, 25 

REAL = 66, 82 .... GUESSED = 65, 81
REAL = 49, 58 .... GUESSED = 50, 58
REAL = 98, 81 .... GUESSED = 96, 81
REAL = 0, 97 .... GUESSED = 69, 62
REAL = 32, 33 .... GUESSED = 32, 33
REAL = 74, 72 .... GUESSED = 73, 70
REAL = 22, 34 .... GUESSED = 21, 33
REAL = 113, 44 .... GUESSED = 112, 42
REAL = 84, 88 .... GUESSED = 83, 87
REAL = 93, 90 .... GUESSED = 93, 90
REAL = 76, 96 .... GUESSED = 74, 95
REAL = 122, 38 .... GUESSED = 121, 39
REAL = 10, 79 .... GUESSED = 11, 79
REAL = 80, 42 .... GUESSED = 79, 41
REAL = 125, 57 .... GUESSED = 123, 55
REAL = 109, 83 .... GUESSED = 107, 82
REAL = 96, 88 .... GUESSED = 95, 88
REAL = 78, 1 .... GUESSED = 78, 0
REAL = 46, 105 .... GUESSED = 45, 104
REAL = 97, 20 .... GUESSED = 96, 20
REAL = 50, 108 .... GUESSED = 50, 108
REAL = 124, 9 .... GUESSED = 73, 73
REAL = 119, 120 .... GUESSED = 114, 115
REAL = 121, 58 .... GUESSED = 120, 57
REAL = 20, 26 .... GUESSED = 20, 26
REAL = 26, 46 .... GUESSED = 26, 46
REAL = 91, 125 .... GUESSED = 91, 125
REAL = 87, 

REAL = 89, 84 .... GUESSED = 87, 83
REAL = 21, 3 .... GUESSED = 21, 4
REAL = 10, 28 .... GUESSED = 11, 29
REAL = 127, 120 .... GUESSED = 102, 95
REAL = 83, 95 .... GUESSED = 83, 94
REAL = 41, 25 .... GUESSED = 42, 26
REAL = 65, 6 .... GUESSED = 66, 8
REAL = 80, 117 .... GUESSED = 79, 116
REAL = 91, 83 .... GUESSED = 90, 82
REAL = 78, 55 .... GUESSED = 77, 55
REAL = 120, 36 .... GUESSED = 119, 36
REAL = 53, 15 .... GUESSED = 53, 15
REAL = 22, 94 .... GUESSED = 22, 94
REAL = 85, 19 .... GUESSED = 86, 20
REAL = 9, 112 .... GUESSED = 68, 71
REAL = 26, 39 .... GUESSED = 26, 38
REAL = 36, 111 .... GUESSED = 36, 110
REAL = 22, 12 .... GUESSED = 24, 13
REAL = 95, 106 .... GUESSED = 94, 106
REAL = 77, 94 .... GUESSED = 76, 93
REAL = 110, 61 .... GUESSED = 109, 59
REAL = 91, 8 .... GUESSED = 90, 7
REAL = 69, 65 .... GUESSED = 68, 65
REAL = 21, 1 .... GUESSED = 22, 2
REAL = 61, 81 .... GUESSED = 61, 81
REAL = 31, 8 .... GUESSED = 30, 9
REAL = 87, 71 .... GUESSED = 86, 70
REAL = 42, 11 .... GUESSE

REAL = 78, 86 .... GUESSED = 77, 84
REAL = 32, 67 .... GUESSED = 32, 67
REAL = 30, 120 .... GUESSED = 29, 120
REAL = 86, 3 .... GUESSED = 85, 4
REAL = 114, 67 .... GUESSED = 114, 67
REAL = 41, 38 .... GUESSED = 41, 38
REAL = 23, 92 .... GUESSED = 23, 92
REAL = 23, 125 .... GUESSED = 74, 74
REAL = 95, 72 .... GUESSED = 94, 71
REAL = 96, 116 .... GUESSED = 95, 116
REAL = 106, 36 .... GUESSED = 105, 36
REAL = 106, 91 .... GUESSED = 105, 91
REAL = 56, 38 .... GUESSED = 56, 37
REAL = 109, 89 .... GUESSED = 109, 90
REAL = 116, 31 .... GUESSED = 114, 30
REAL = 49, 45 .... GUESSED = 49, 45
REAL = 49, 95 .... GUESSED = 47, 93
REAL = 106, 119 .... GUESSED = 9, 21
REAL = 91, 107 .... GUESSED = 90, 106
REAL = 122, 126 .... GUESSED = 96, 101
REAL = 34, 22 .... GUESSED = 33, 21
REAL = 114, 5 .... GUESSED = 70, 70
REAL = 19, 88 .... GUESSED = 19, 88
REAL = 32, 3 .... GUESSED = 32, 2
REAL = 124, 60 .... GUESSED = 123, 59
REAL = 107, 90 .... GUESSED = 106, 90
REAL = 37, 44 .... GUESSED = 37, 44
REAL = 

REAL = 107, 107 .... GUESSED = 105, 107
REAL = 19, 32 .... GUESSED = 19, 33
REAL = 41, 13 .... GUESSED = 40, 15
REAL = 86, 28 .... GUESSED = 85, 27
REAL = 67, 46 .... GUESSED = 66, 45
REAL = 126, 112 .... GUESSED = 122, 107
REAL = 104, 108 .... GUESSED = 104, 108
REAL = 61, 22 .... GUESSED = 61, 24
REAL = 124, 29 .... GUESSED = 122, 29
REAL = 6, 44 .... GUESSED = 7, 43
REAL = 60, 41 .... GUESSED = 59, 40
REAL = 87, 43 .... GUESSED = 86, 43
REAL = 52, 116 .... GUESSED = 52, 114
REAL = 111, 91 .... GUESSED = 110, 91
REAL = 13, 15 .... GUESSED = 14, 16
REAL = 12, 11 .... GUESSED = 12, 12
REAL = 49, 101 .... GUESSED = 48, 100
REAL = 51, 90 .... GUESSED = 50, 89
REAL = 96, 34 .... GUESSED = 96, 33
REAL = 11, 91 .... GUESSED = 11, 90
REAL = 87, 96 .... GUESSED = 87, 96
REAL = 84, 93 .... GUESSED = 83, 92
REAL = 79, 111 .... GUESSED = 79, 110
REAL = 10, 47 .... GUESSED = 11, 48
REAL = 82, 69 .... GUESSED = 82, 67
REAL = 46, 58 .... GUESSED = 47, 58
REAL = 59, 32 .... GUESSED = 59, 32
REAL = 1

KeyboardInterrupt: 

In [ ]:
## test some stuff

In [136]:
for data in dataloader:
    conv1 = nn.Conv2d(1, 10, 5)
    pool = nn.MaxPool2d(2, 2)
    conv2 = nn.Conv2d(10, 10, 5)
    conv3 = nn.Conv2d(10, 10, 3)
    fc1 = nn.Linear(10 * 13 * 13, 250)
    fc2 = nn.Linear(250, 200)
    fc3 = nn.Linear(200, 100)

    x = data["image"]
    print(x.shape)
    x = pool(F.relu(conv1(x)))
    print(x.shape)
    x = pool(F.relu(conv2(x)))
    print(x.shape)
    x = pool(F.relu(conv3(x)))
    print(x.shape)
    x = x.view(-1, 10 * 13 * 13)
    x = F.relu(fc1(x))
    x = F.relu(fc2(x))
    x = fc3(x)
    print(x.shape)
    print(data["radius"])
    print(data["radius"].shape)
    break

torch.Size([16, 1, 128, 128])
torch.Size([16, 10, 62, 62])
torch.Size([16, 10, 29, 29])
torch.Size([16, 10, 13, 13])
torch.Size([16, 100])
tensor([[ 5],
        [82],
        [55],
        [31],
        [52],
        [58],
        [74],
        [33],
        [17],
        [49],
        [13],
        [80],
        [19],
        [46],
        [72],
        [18]])
torch.Size([16, 1])


In [140]:
torch.max(x, 1)[1][0]

tensor(9)